In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
gpu_device_name = tf.test.gpu_device_name()
print(gpu_device_name)

# 显存限制

默认情况下，TensorFlow会映射该进程可见的所有GPU的几乎所有GPU内存（取决于CUDA_VISIBLE_DEVICES）。 这样做是为了通过减少内存碎片来更有效地使用设备上相对宝贵的GPU内存资源。 要将TensorFlow限制为一组特定的GPU，我们使用tf.config.experimental.set_visible_devices方法。

In [ ]:
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   # Restrict TensorFlow to only use the first GPU
#     try:
#         tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
#         logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#         print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
#     except RuntimeError as e:
#         # Visible devices must be set before GPUs have been initialized
#         print(e)

在某些情况下，希望该过程仅分配可用内存的子集，或者仅增加该过程所需的内存使用量。 TensorFlow提供了两种控制方法。

第一种选择是通过调用tf.config.experimental.set_memory_growth来打开内存增长，tf.config.experimental.set_memory_growth尝试仅分配运行时分配所需的GPU内存：它开始分配的内存很少，并且随着程序的运行和 需要更多的GPU内存，我们扩展了分配给TensorFlow进程的GPU内存区域。 请注意，我们不会释放内存，因为它可能导致内存碎片。 要打开特定GPU的内存增长，请在分配任何张量或执行任何操作之前使用以下代码。

In [ ]:
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#     try:
#     # Currently, memory growth needs to be the same across GPUs
#         for gpu in gpus:
#             tf.config.experimental.set_memory_growth(gpu, True)
#             logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#             print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#     except RuntimeError as e:
#         # Memory growth must be set before GPUs have been initialized
#         print(e)

启用此选项的另一种方法是将环境变量TF_FORCE_GPU_ALLOW_GROWTH设置为true。 此配置是特定于平台的。

第二种方法是使用tf.config.experimental.set_virtual_device_configuration配置虚拟GPU设备，并对要在GPU上分配的总内存设置硬限制。

如果您想真正绑定TensorFlow进程可用的GPU内存量，这将很有用。 当GPU与其他应用程序（例如工作站GUI）共享GPU时，这是本地开发的常见做法。`

今天我们来做NLP（自然语言处理）中Sequence2Sequence的任务。其中Sequence2Sequence任务在生活中最常见的应用场景就是机器翻译。除了机器翻译之外，现在很流行的对话机器人任务，摘要生成任务都是典型的Sequence2Sequence。Sequence2Sequence的难点在于模型需要干两件比较难的事情：

语义理解（NLU:Natural Language Understanding）：模型必须理解输入的句子。
句子生成(NLG:Natural Language Generation)：模型生成的句子需符合句法，不能是人类觉得不通顺的句子。
想想看，让模型理解输入句子的语义已经很困难了，还得需要它返回一个符合人类造句句法的序列。不过还是那句话，没有什么是深度学习不能解决的，如果有，当我没说上句话。

basic encoder-decoder ：将encode出来的编码全部丢给decode每个step

encoder-decoder with feedback ：将encode出来的编码只喂给decode的初始step，在解码器端，需将每个step的输出，输入给下一个step。

encoder-decoder with peek：1和2的组合，不仅将encode出来的编码全部丢给decode每个step，在解码器端，也将每个step的输出，输入给下一个step。
4.encoder-decoder with attention:将3模型的encode端做了一个小小的改进，加入了attention机制，简单来说，就是对encode端每个step的输入做了一个重要性打分。



本次实验采用的是basic encoder-decoder架构，下面开始实战部分。

对对联实战 数据加载 数据样式如下图所示是一对对联。模型的输入时一句"晚 风 摇 树 树 还 挺"，需要模型生成" 晨 露 润 花 花 更 红"。这个数据集有个特点，就是输入输出是等长的，序列标注算法在这个数据集上也是适用的。

/device:GPU:0


# 显存限制

默认情况下，TensorFlow会映射该进程可见的所有GPU的几乎所有GPU内存（取决于CUDA_VISIBLE_DEVICES）。 这样做是为了通过减少内存碎片来更有效地使用设备上相对宝贵的GPU内存资源。 要将TensorFlow限制为一组特定的GPU，我们使用tf.config.experimental.set_visible_devices方法。

In [4]:
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   # Restrict TensorFlow to only use the first GPU
#     try:
#         tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
#         logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#         print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
#     except RuntimeError as e:
#         # Visible devices must be set before GPUs have been initialized
#         print(e)

在某些情况下，希望该过程仅分配可用内存的子集，或者仅增加该过程所需的内存使用量。 TensorFlow提供了两种控制方法。

第一种选择是通过调用tf.config.experimental.set_memory_growth来打开内存增长，tf.config.experimental.set_memory_growth尝试仅分配运行时分配所需的GPU内存：它开始分配的内存很少，并且随着程序的运行和 需要更多的GPU内存，我们扩展了分配给TensorFlow进程的GPU内存区域。 请注意，我们不会释放内存，因为它可能导致内存碎片。 要打开特定GPU的内存增长，请在分配任何张量或执行任何操作之前使用以下代码。

In [5]:
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#     try:
#     # Currently, memory growth needs to be the same across GPUs
#         for gpu in gpus:
#             tf.config.experimental.set_memory_growth(gpu, True)
#             logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#             print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#     except RuntimeError as e:
#         # Memory growth must be set before GPUs have been initialized
#         print(e)

启用此选项的另一种方法是将环境变量TF_FORCE_GPU_ALLOW_GROWTH设置为true。 此配置是特定于平台的。

第二种方法是使用tf.config.experimental.set_virtual_device_configuration配置虚拟GPU设备，并对要在GPU上分配的总内存设置硬限制。

如果您想真正绑定TensorFlow进程可用的GPU内存量，这将很有用。 当GPU与其他应用程序（例如工作站GUI）共享GPU时，这是本地开发的常见做法。`

今天我们来做NLP（自然语言处理）中Sequence2Sequence的任务。其中Sequence2Sequence任务在生活中最常见的应用场景就是机器翻译。除了机器翻译之外，现在很流行的对话机器人任务，摘要生成任务都是典型的Sequence2Sequence。Sequence2Sequence的难点在于模型需要干两件比较难的事情：

语义理解（NLU:Natural Language Understanding）：模型必须理解输入的句子。
句子生成(NLG:Natural Language Generation)：模型生成的句子需符合句法，不能是人类觉得不通顺的句子。
想想看，让模型理解输入句子的语义已经很困难了，还得需要它返回一个符合人类造句句法的序列。不过还是那句话，没有什么是深度学习不能解决的，如果有，当我没说上句话。

basic encoder-decoder ：将encode出来的编码全部丢给decode每个step

encoder-decoder with feedback ：将encode出来的编码只喂给decode的初始step，在解码器端，需将每个step的输出，输入给下一个step。

encoder-decoder with peek：1和2的组合，不仅将encode出来的编码全部丢给decode每个step，在解码器端，也将每个step的输出，输入给下一个step。
4.encoder-decoder with attention:将3模型的encode端做了一个小小的改进，加入了attention机制，简单来说，就是对encode端每个step的输入做了一个重要性打分。



本次实验采用的是basic encoder-decoder架构，下面开始实战部分。

对对联实战 数据加载 数据样式如下图所示是一对对联。模型的输入时一句"晚 风 摇 树 树 还 挺"，需要模型生成" 晨 露 润 花 花 更 红"。这个数据集有个特点，就是输入输出是等长的，序列标注算法在这个数据集上也是适用的。